In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline

In [2]:
BSinfo = pd.read_csv('../../Dataset/BSinfo.csv')
CLdata = pd.read_csv('../../Dataset/CLdata.csv')
ECdata = pd.read_csv('../../Dataset/ECdata.csv')
sample_submission = pd.read_csv('../../Dataset/SampleSubmission.csv')

In [3]:
ECdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92629 entries, 0 to 92628
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    92629 non-null  object 
 1   BS      92629 non-null  object 
 2   Energy  92629 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.1+ MB


In [4]:
BSinfo.head()

,BS,CellName,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,B_0,Cell0,Type1,Mode2,365.0,20,4,6.875934
1,B_1,Cell0,Type2,Mode2,532.0,20,4,6.875934
2,B_2,Cell0,Type1,Mode2,365.0,20,4,6.875934
3,B_3,Cell0,Type2,Mode2,532.0,20,4,6.875934
4,B_4,Cell0,Type2,Mode2,532.0,20,4,6.875934


In [5]:
ECdata.head()

,Time,BS,Energy
0,1/1/2023 1:00,B_0,64.275037
1,1/1/2023 2:00,B_0,55.904335
2,1/1/2023 3:00,B_0,57.698057
3,1/1/2023 4:00,B_0,55.156951
4,1/1/2023 5:00,B_0,56.053812


In [6]:
sample_submission.head()

,Time,BS,Energy,w
0,2023-01-01 06:00:00,B_0,NaN,1
1,2023-01-01 11:00:00,B_0,NaN,1
2,2023-01-01 12:00:00,B_0,NaN,1
3,2023-01-01 13:00:00,B_0,NaN,1
4,2023-01-01 23:00:00,B_0,NaN,1


In [7]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26139 entries, 0 to 26138
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    26139 non-null  object 
 1   BS      26139 non-null  object 
 2   Energy  0 non-null      float64
 3   w       26139 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 817.0+ KB


In [8]:
ECdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92629 entries, 0 to 92628
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    92629 non-null  object 
 1   BS      92629 non-null  object 
 2   Energy  92629 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.1+ MB


In [9]:
sample_submission['Time'] = pd.to_datetime(sample_submission['Time'])
ECdata['Time'] = pd.to_datetime(ECdata['Time'])


In [10]:
sample_submission.head()

,Time,BS,Energy,w
0,2023-01-01 06:00:00,B_0,NaN,1
1,2023-01-01 11:00:00,B_0,NaN,1
2,2023-01-01 12:00:00,B_0,NaN,1
3,2023-01-01 13:00:00,B_0,NaN,1
4,2023-01-01 23:00:00,B_0,NaN,1


In [11]:
ECdata.head()

,Time,BS,Energy
0,2023-01-01 01:00:00,B_0,64.275037
1,2023-01-01 02:00:00,B_0,55.904335
2,2023-01-01 03:00:00,B_0,57.698057
3,2023-01-01 04:00:00,B_0,55.156951
4,2023-01-01 05:00:00,B_0,56.053812


In [12]:
# concatinate the ECdata and sample_submission
FullData = pd.concat([ECdata, sample_submission], axis=0)
FullData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118768 entries, 0 to 26138
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Time    118768 non-null  datetime64[ns]
 1   BS      118768 non-null  object        
 2   Energy  92629 non-null   float64       
 3   w       26139 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 4.5+ MB


In [13]:
# split the Time column into Date, Time and Hour
FullData['Date'] = FullData['Time'].dt.date
FullData['Time1'] = FullData['Time'].dt.time
FullData['Hour'] = FullData['Time'].apply(lambda x: x.hour)
FullData.head()

,Time,BS,Energy,w,Date,Time1,Hour
0,2023-01-01 01:00:00,B_0,64.275037,NaN,2023-01-01,01:00:00,1
1,2023-01-01 02:00:00,B_0,55.904335,NaN,2023-01-01,02:00:00,2
2,2023-01-01 03:00:00,B_0,57.698057,NaN,2023-01-01,03:00:00,3
3,2023-01-01 04:00:00,B_0,55.156951,NaN,2023-01-01,04:00:00,4
4,2023-01-01 05:00:00,B_0,56.053812,NaN,2023-01-01,05:00:00,5


In [14]:
sample_submission.BS.unique()

# what are the BSs in the sample_submission which are not in the 
sample_submission[~sample_submission.BS.isin(ECdata.BS)].BS.nunique()

97

In [15]:
sample_submission[~sample_submission.BS.isin(BSinfo.BS)].BS.unique()

array([], dtype=object)

In [16]:
BSinfo[~BSinfo.BS.isin(sample_submission.BS)].BS.unique()

array(['B_924', 'B_946'], dtype=object)

In [17]:
sample_submission.head()

,Time,BS,Energy,w
0,2023-01-01 06:00:00,B_0,NaN,1
1,2023-01-01 11:00:00,B_0,NaN,1
2,2023-01-01 12:00:00,B_0,NaN,1
3,2023-01-01 13:00:00,B_0,NaN,1
4,2023-01-01 23:00:00,B_0,NaN,1


In [18]:
BSinfo.head()

,BS,CellName,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,B_0,Cell0,Type1,Mode2,365.0,20,4,6.875934
1,B_1,Cell0,Type2,Mode2,532.0,20,4,6.875934
2,B_2,Cell0,Type1,Mode2,365.0,20,4,6.875934
3,B_3,Cell0,Type2,Mode2,532.0,20,4,6.875934
4,B_4,Cell0,Type2,Mode2,532.0,20,4,6.875934


In [19]:
CLdata.head()

,Time,BS,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6
0,1/1/2023 1:00,B_0,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0
1,1/1/2023 2:00,B_0,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0
2,1/1/2023 3:00,B_0,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0
3,1/1/2023 4:00,B_0,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0
4,1/1/2023 5:00,B_0,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0


In [20]:
sample_submission[~sample_submission.BS.isin(CLdata.BS)].BS.unique()

array([], dtype=object)

In [21]:
CLdata.ESMode6.value_counts() # esmode 1,2,3,6 are important

ESMode6
0.000000    118245
0.924341         6
0.924342         5
0.924337         3
0.924339         3
             ...  
0.701364         1
0.686719         1
0.708874         1
0.774581         1
0.537125         1
Name: count, Length: 7303, dtype: int64

In [22]:
# merge the CLdata and BSinfo using (BS,CellName)
CL_BS = pd.merge(CLdata, BSinfo, on=['BS', 'CellName'])
CL_BS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125575 entries, 0 to 125574
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Time       125575 non-null  object 
 1   BS         125575 non-null  object 
 2   CellName   125575 non-null  object 
 3   load       125575 non-null  float64
 4   ESMode1    125575 non-null  float64
 5   ESMode2    125575 non-null  float64
 6   ESMode3    125575 non-null  float64
 7   ESMode4    125575 non-null  int64  
 8   ESMode5    125575 non-null  float64
 9   ESMode6    125575 non-null  float64
 10  RUType     125575 non-null  object 
 11  Mode       125575 non-null  object 
 12  Frequency  125575 non-null  float64
 13  Bandwidth  125575 non-null  int64  
 14  Antennas   125575 non-null  int64  
 15  TXpower    125575 non-null  float64
dtypes: float64(8), int64(3), object(5)
memory usage: 15.3+ MB


In [23]:
CL_BS.head()

,Time,BS,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,1/1/2023 1:00,B_0,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
1,1/1/2023 2:00,B_0,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
2,1/1/2023 3:00,B_0,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
3,1/1/2023 4:00,B_0,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
4,1/1/2023 5:00,B_0,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934


In [24]:
# remove ESMode4, ESMode5 columns
CL_BS.drop(['ESMode4', 'ESMode5'], axis=1, inplace=True)
CL_BS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125575 entries, 0 to 125574
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Time       125575 non-null  object 
 1   BS         125575 non-null  object 
 2   CellName   125575 non-null  object 
 3   load       125575 non-null  float64
 4   ESMode1    125575 non-null  float64
 5   ESMode2    125575 non-null  float64
 6   ESMode3    125575 non-null  float64
 7   ESMode6    125575 non-null  float64
 8   RUType     125575 non-null  object 
 9   Mode       125575 non-null  object 
 10  Frequency  125575 non-null  float64
 11  Bandwidth  125575 non-null  int64  
 12  Antennas   125575 non-null  int64  
 13  TXpower    125575 non-null  float64
dtypes: float64(7), int64(2), object(5)
memory usage: 13.4+ MB


In [25]:
# change the Time column to datetime
CL_BS['Time'] = pd.to_datetime(CL_BS['Time'])

# split the Time column into Date, Time and Hour
CL_BS['Date'] = CL_BS['Time'].dt.date
CL_BS['Time1'] = CL_BS['Time'].dt.time
CL_BS['Hour'] = CL_BS['Time'].apply(lambda x: x.hour)
CL_BS.head()

,Time,BS,CellName,load,ESMode1,ESMode2,ESMode3,ESMode6,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower,Date,Time1,Hour
0,2023-01-01 01:00:00,B_0,Cell0,0.487936,0.0,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934,2023-01-01,01:00:00,1
1,2023-01-01 02:00:00,B_0,Cell0,0.344468,0.0,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934,2023-01-01,02:00:00,2
2,2023-01-01 03:00:00,B_0,Cell0,0.193766,0.0,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934,2023-01-01,03:00:00,3
3,2023-01-01 04:00:00,B_0,Cell0,0.222383,0.0,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934,2023-01-01,04:00:00,4
4,2023-01-01 05:00:00,B_0,Cell0,0.175436,0.0,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934,2023-01-01,05:00:00,5
